# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [19]:
import pandas as pd
import numpy as np
from scipy import stats

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
salaries['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [12]:
salaries['Department'].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [133]:
hourly_rates = salaries['Hourly Rate'][salaries['Salary or Hourly']=='Hourly']
t, p_val = stats.ttest_1samp(hourly_rates, 30)

print(f'We reject the null hypothesis with a p_value of {round(p_val, 3)}')
hourly_rates.mean()

We reject the null hypothesis with a p_value of 0.0


32.78855771628024

**Conclusion**:The mean hourly wages are 32.78 dollars and are significantly different from 30.

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [139]:
def one_side(p_val, up = True):
    if up==True:
        if p_val/2 < 0.05 and t>0:
            print(f'We reject the null hypothesis with a p_value of {round(p_val/2, 3)}')
        else: print(f'We cannot reject the null hypothesis with a p_value of {round(p_val/2, 3)}')
    elif up==False:
        if p_val/2 < 0.05 and t<0:
            print(f'We reject the null hypothesis with a p_value of {round(p_val/2, 3)}')
        else: print(f'We cannot reject the null hypothesis with a p_value of {round(p_val/2, 3)}')


police = salaries['Annual Salary'][(salaries['Salary or Hourly']=='Salary') & (salaries['Department']=='POLICE')]
t, p_val = stats.ttest_1samp(police, 86000)

#For it to be significant on one side, we divide the p-value by 2 and check that the statistic is positive:

one_side_up(p_val)
print(t,p_val/2)
print(police.mean())

We reject the null hypothesis with a p_value of 0.001
3.081997005712994 0.0010301701775482569
86486.41450313339


**Comments**: The police salaries are significantly higher than 86000 dollars.

Using the `crosstab` function, find the department that has the most hourly workers. 

In [88]:
cross = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'])
cross[cross.Hourly==cross.Hourly.max()]

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [141]:
t, p_val = stats.ttest_1samp(hourly_rates, 35)

one_side(p_val, up=False)
print(t, p_val/2, hourly_rates.mean())

We reject the null hypothesis with a p_value of 0.0
-16.352363851001144 1.8818444649254009e-59 32.78855771628024


**Comments**: The salaries for hourly workers are significantly lower than 35 dollars

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [100]:
ci = stats.t.interval(0.95, len(hourly_rates)-1, hourly_rates.mean(), stats.sem(hourly_rates))
print(f'The 95% confident interval is {ci}')

The 95% confident interval is (32.52345834488425, 33.05365708767623)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [101]:
ci = stats.t.interval(0.95, len(police)-1, police.mean(), stats.sem(police))
print(f'The 95% confident interval is {ci}')

The 95% confident interval is (86177.05631531784, 86795.77269094894)


**Comments**: 95% of the times we run this same test, the mean will be within that interval

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [146]:
from statsmodels.stats.proportion import proportions_ztest

#Counts of workers:
count = len(salaries)
hourly_count = len(hourly_rates)

z, p_val = proportions_ztest(count=hourly_count, nobs=count, value=0.25, alternative='two-sided')
p_val


0.01920825109701381

**Comments**: The number of hourly workers is significantly different than 25.